<a href="https://colab.research.google.com/github/dcolinmorgan/grph/blob/main/smart_dep_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- colab out-of-the-box without gpu <= pandas
- colab out-of-the-box with gpu <= pandas
- colab out-of-the-box with gpu+rapids installed <= pandas + cudf

functionality: compute_cugraph(), featurize(), umap()

In [1]:
!pip install -U -q git+https://github.com/graphistry/pygraphistry.git@dev/depman

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.3/332.3 kB 10.3 MB/s eta 0:00:00


In [2]:
!pip install -q umap_learn dirty_cat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 2.4 MB/s eta 0:00:00


In [3]:
from google.colab import userdata
g_user=userdata.get('g_user')
g_pass=userdata.get('g_pass')

In [4]:
import graphistry
graphistry.register(api=3,protocol="https", server="hub.graphistry.com", username=g_user, password=g_pass) ## key id, secret key

graphistry.__version__
import pandas as pd

In [5]:
%time
df = pd.read_csv('https://gist.githubusercontent.com/silkspace/c7b50d0c03dc59f63c48d68d696958ff/raw/31d918267f86f8252d42d2e9597ba6fc03fcdac2/redteam_50k.csv', index_col=0)
red_team = pd.read_csv('https://gist.githubusercontent.com/silkspace/5cf5a94b9ac4b4ffe38904f20d93edb1/raw/888dabd86f88ea747cf9ff5f6c44725e21536465/redteam_labels.csv', index_col=0)
df['feats'] = df.src_computer + ' ' + df.dst_computer + ' ' + df.auth_type + ' ' + df.logontype
df['feats2'] = df.src_computer + ' ' + df.dst_computer
ndf = df.drop_duplicates(subset=['feats'])
tdf = pd.concat([red_team.reset_index(), ndf.reset_index()])
tdf['node'] = range(len(tdf))
tdf.RED.sum()

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.53 µs


749.0

## out-of-the-box without GPU

In [6]:
%%time

g = graphistry.nodes(tdf, 'node')
g.featurize()

ERROR:graphistry.feature_utils:dirty_cat not found for featurizing/usr/local/lib/python3.10/dist-packages/cudf/utils/_ptxcompiler.py:61: UserWarning: Error getting driver and runtime versions:

stdout:



stderr:

Traceback (most recent call last):
  File "<string>", line 4, in <module>
  File "/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/driver.py", line 295, in __getattr__
    raise CudaSupportError("Error at driver init: \n%s:" %
numba.cuda.cudadrv.error.CudaSupportError: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path of the CUDA driver shared library.
:


Not patching Numba
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.10/dist-packages/cudf/utils/gpu_utils.py:62: UserWarning: Function "cuInit" not found
  warnings.warn(str(e))
ERROR:graphistry.utils.dep_manager:cudf installed but misconfigured/usr/local/lib/python3.10/dist-pac

CPU times: user 1.02 s, sys: 146 ms, total: 1.17 s
Wall time: 10.4 s


{'bindings': {'edges': None,
  'nodes':           index    time             src_domain src_computer dst_computer  \
  0             0  150885              U620@DOM1       C17693        C1003   
  1             1  151036              U748@DOM1       C17693         C305   
  2             2  151648              U748@DOM1       C17693         C728   
  3             3  151993             U6115@DOM1       C17693        C1173   
  4             4  153792              U636@DOM1       C17693         C294   
  ...         ...     ...                    ...          ...          ...   
  19008   8463107   48263           C11843$@DOM1       C11843         C528   
  19009  14394630   77937            C8470$@DOM1        C8470         C528   
  19010  33398153  173300             C716$@DOM1         C716         C716   
  19011  18353851  102472             U7365@DOM1       C16126         C586   
  19012  27372458  141156  NETWORK SERVICE@C6215        C6215        C6215   
  
                       

In [7]:
%%time

g = graphistry.nodes(tdf, 'node')
g1 = g.umap(engine='umap_learn')

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/cupy/__init__.py", line 17, in <module>
    from cupy import _core  # NOQA
  File "/usr/local/lib/python3.10/dist-packages/cupy/_core/__init__.py", line 3, in <module>
    from cupy._core import core  # NOQA
ImportError: libcuda.so.1: cannot open shared object file: No such file or directory

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/graphistry/utils/lazy_import.py", line 11, in lazy_cudf_import
    import cudf  # type: ignore
  File "/usr/local/lib/python3.10/dist-packages/cudf/__init__.py", line 12, in <module>
    import cupy
  File "/usr/local/lib/python3.10/dist-packages/cupy/__init__.py", line 19, in <module>
    raise ImportError(f'''
ImportError: 
Failed to import CuPy.

If you installed CuPy via wheels (cupy-cudaXXX or cupy-rocm-X-X), make sure that the package matches with the version 

CPU times: user 4min 28s, sys: 2.32 s, total: 4min 30s
Wall time: 3min 16s


In [8]:
g1.plot()

## out-of-the-box with GPU

In [6]:
%%time

g = graphistry.nodes(tdf, 'node')
g.featurize()

ERROR:graphistry.feature_utils:dirty_cat not found for featurizingERROR:graphistry.feature_utils:dirty_cat not found for featurizing

CPU times: user 382 ms, sys: 73.3 ms, total: 455 ms
Wall time: 910 ms


{'bindings': {'edges': None,
  'nodes':           index    time             src_domain src_computer dst_computer  \
  0             0  150885              U620@DOM1       C17693        C1003   
  1             1  151036              U748@DOM1       C17693         C305   
  2             2  151648              U748@DOM1       C17693         C728   
  3             3  151993             U6115@DOM1       C17693        C1173   
  4             4  153792              U636@DOM1       C17693         C294   
  ...         ...     ...                    ...          ...          ...   
  19008   8463107   48263           C11843$@DOM1       C11843         C528   
  19009  14394630   77937            C8470$@DOM1        C8470         C528   
  19010  33398153  173300             C716$@DOM1         C716         C716   
  19011  18353851  102472             U7365@DOM1       C16126         C586   
  19012  27372458  141156  NETWORK SERVICE@C6215        C6215        C6215   
  
                       

In [7]:
%%time

g = graphistry.nodes(tdf, 'node')
g2 = g.umap(engine='umap_learn')

ERROR:graphistry.feature_utils:dirty_cat not found for featurizingWARNING:graphistry.umap_utils:* Ignoring target column of shape (19762, 0) in UMAP fit, as it is not one dimensional

CPU times: user 4min 5s, sys: 1.38 s, total: 4min 6s
Wall time: 2min 45s


In [8]:
g2.plot()

## install RAPIDS

In [ ]:
# !pip install -q --extra-index-url=https://pypi.nvidia.com cuml-cu12==24.8.0 cugraph-cu12==24.8.0  #cudf-cu12==24.8.1 already installed & confuses system
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py


In [11]:
import cuml

In [12]:
import cugraph

In [23]:
%%time
df = pd.read_csv('https://raw.githubusercontent.com/graphistry/pygraphistry/master/demos/data/lesmiserables.csv')
g = graphistry.edges(df, df.columns[0], df.columns[1])
g._edges.sample(5)
g2 = g.compute_cugraph('pagerank')
g2._nodes.sample(5)

CPU times: user 925 ms, sys: 191 ms, total: 1.12 s
Wall time: 1.53 s


,id,pagerank
26,Marguerite,0.008150
33,Favourite,0.010886
11,Bahorel,0.005785
4,BaronessT,0.004004
42,Bamatabois,0.011859


In [14]:
%%time

g = graphistry.nodes(tdf, 'node')
g.featurize()

ERROR:graphistry.feature_utils:dirty_cat not found for featurizingERROR:graphistry.feature_utils:dirty_cat not found for featurizing

CPU times: user 84.1 ms, sys: 4.9 ms, total: 89 ms
Wall time: 212 ms


{'bindings': {'edges': None,
  'nodes':           index    time             src_domain src_computer dst_computer  \
  0             0  150885              U620@DOM1       C17693        C1003   
  1             1  151036              U748@DOM1       C17693         C305   
  2             2  151648              U748@DOM1       C17693         C728   
  3             3  151993             U6115@DOM1       C17693        C1173   
  4             4  153792              U636@DOM1       C17693         C294   
  ...         ...     ...                    ...          ...          ...   
  19008   8463107   48263           C11843$@DOM1       C11843         C528   
  19009  14394630   77937            C8470$@DOM1        C8470         C528   
  19010  33398153  173300             C716$@DOM1         C716         C716   
  19011  18353851  102472             U7365@DOM1       C16126         C586   
  19012  27372458  141156  NETWORK SERVICE@C6215        C6215        C6215   
  
                       

In [17]:
%%time

g = graphistry.nodes(tdf, 'node')
g3 = g.umap(engine='cuml')

ERROR:graphistry.feature_utils:dirty_cat not found for featurizingWARNING:graphistry.util.memoization:! Failed umap speedup attempt. Continuing without memoization speedups.WARNING:graphistry.umap_utils:* Ignoring target column of shape (19762, 0) in UMAP fit, as it is not one dimensional

CPU times: user 9.73 s, sys: 636 ms, total: 10.4 s
Wall time: 11.7 s
